In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'

In [3]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from collections import Counter

In [4]:
train_df = pd.read_csv(raw_data_dir+'sales_train_evaluation.csv')
prices_df = pd.read_csv(raw_data_dir+'sell_prices.csv')
calendar_df = pd.read_csv(raw_data_dir+'calendar.csv')

### Item

In [37]:
mean_by_item = train_df.drop(['id', 'cat_id', 'dept_id', 'store_id', 'state_id'], axis=1)
mean_by_item = mean_by_item.groupby([mean_by_item['item_id']]).mean()
mean_by_item = mean_by_item.reset_index()
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [38]:
a = mean_by_item.drop(['item_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.5,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3
4,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,4.1,...,1.5,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [39]:
# euclidean
tskm_10 = TimeSeriesKMeans(n_clusters=10, verbose=False, random_state=123, n_jobs=-1)
y_pred_10 = tskm_10.fit_predict(a)
print('y_pred_10: ', sorted(Counter(y_pred_10).items()))

y_pred_10:  [(0, 2108), (1, 8), (2, 3), (3, 16), (4, 655), (5, 1), (6, 190), (7, 1), (8, 35), (9, 32)]


In [40]:
tskm_eu = pd.DataFrame({'tskm_10': y_pred_10})
tskm_eu

,tskm_10
0,0
1,0
2,0
3,9
4,4
...,...
3044,0
3045,0
3046,0
3047,0


In [41]:
mean_by_item = pd.concat([mean_by_item, tskm_eu], axis = 1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,9
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0


In [42]:
div_clu0 = mean_by_item[mean_by_item['tskm_10'] == 0]
div_clu0 = div_clu0.drop(['item_id', 'tskm_10'], axis=1)
div_clu0

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,1.0,...,0.7,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9
1,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,0.4,...,0.5,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4
2,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,0.6,...,0.7,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.1,0.5,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,0.5,...,0.5,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4
3046,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,0.1,...,0.2,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1


In [43]:
tskm_5 = TimeSeriesKMeans(n_clusters=5, verbose=False, random_state=123, n_jobs=-1)
y_pred_5 = tskm_5.fit_predict(div_clu0)
print('y_pred_5: ', sorted(Counter(y_pred_5).items()))

y_pred_5:  [(0, 296), (1, 496), (2, 814), (3, 164), (4, 338)]


In [44]:
tskm_5 = pd.DataFrame({'tskm_5': y_pred_5})
div_clu0 = mean_by_item[mean_by_item['tskm_10'] == 0]
div_clu0 = div_clu0.reset_index()
div_clu0 = pd.concat([div_clu0, tskm_5], axis=1)
div_clu0

,index,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10,tskm_5
0,0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0,4
1,1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0,1
2,2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0,4
3,6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,0,2
4,7,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0,4
2104,3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0,0
2105,3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0,2
2106,3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,0,2


In [45]:
# 클러스터 레이블 중복 방지를 위해 레이블 넘버 변경
div_clu0.loc[div_clu0['tskm_5'] == 1, 'tskm_5'] = 10
div_clu0.loc[div_clu0['tskm_5'] == 2, 'tskm_5'] = 11
div_clu0.loc[div_clu0['tskm_5'] == 3, 'tskm_5'] = 12
div_clu0.loc[div_clu0['tskm_5'] == 4, 'tskm_5'] = 13
div_clu0 = div_clu0.set_index('index')
div_clu0

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10,tskm_5
index,,,,,,,,,,,,,,,,,,,,,
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,0,13
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,0,10
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,0,13
6,FOODS_1_008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.3,0.2,0.0,0.3,0.2,0.1,0.0,0.4,0,11
7,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,1.9,1.5,0.2,0.7,1.0,1.0,1.3,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,0,13
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0,0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,0,11


In [46]:
b = pd.concat([mean_by_item, div_clu0['tskm_5']], axis = 1)
b.loc[b['tskm_10'] == 0, 'tskm_10'] = b['tskm_5']
mean_by_item = b.drop(['tskm_5'], axis=1)
mean_by_item

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10
0,FOODS_1_001,0.6,0.6,0.4,0.6,0.7,1.8,1.0,0.4,1.1,...,0.8,0.8,0.4,0.7,0.7,0.5,0.7,0.5,0.9,13.0
1,FOODS_1_002,0.4,0.5,0.7,0.4,0.3,0.4,0.1,0.7,0.2,...,0.8,0.7,0.2,0.6,0.5,0.0,0.6,0.6,0.4,10.0
2,FOODS_1_003,1.4,0.8,0.3,0.6,0.3,0.8,1.3,1.0,1.1,...,0.8,0.6,1.1,0.9,0.7,0.7,1.0,0.6,0.5,13.0
3,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.4,10.7,8.1,7.3,6.2,7.1,7.5,8.3,9.3,9.0
4,FOODS_1_005,3.4,3.2,1.3,2.0,1.0,2.1,1.8,2.0,2.5,...,1.6,2.5,0.9,0.7,2.5,1.9,2.5,1.7,1.9,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,0.5,0.4,0.1,0.3,0.2,0.4,0.2,0.8,0.7,...,0.7,1.1,0.4,0.8,0.3,0.4,0.5,0.4,0.7,13.0
3045,HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.3,0.6,0.3,0.0,0.9,0.3,0.4,0.0
3046,HOUSEHOLD_2_514,0.4,0.8,0.2,0.1,0.1,0.2,0.3,0.8,0.2,...,0.2,0.2,0.6,0.1,0.0,0.1,0.4,0.3,0.3,11.0
3047,HOUSEHOLD_2_515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.5,0.5,0.1,0.2,0.3,0.0,0.3,0.1,11.0


In [47]:
dcol=[]
for i in list(mean_by_item):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_item = mean_by_item.drop(dcol, axis=1)
mean_by_item = mean_by_item.astype({'tskm_10':'int'})
mean_by_item

,item_id,tskm_10
0,FOODS_1_001,13
1,FOODS_1_002,10
2,FOODS_1_003,13
3,FOODS_1_004,9
4,FOODS_1_005,4
...,...,...
3044,HOUSEHOLD_2_512,13
3045,HOUSEHOLD_2_513,0
3046,HOUSEHOLD_2_514,11
3047,HOUSEHOLD_2_515,11


In [48]:
train_clu = pd.merge(train_df, mean_by_item, on=['item_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_10
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,0
1,HOBBIES_1_001_CA_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,2,0,2,2,0,2,0,1,0
2,HOBBIES_1_001_CA_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,6,0,1,0,2,1,0,1,0,0
3,HOBBIES_1_001_CA_4_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,0,3,1,1,1,0,1,2,2,0
4,HOBBIES_1_001_TX_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,2,1,0,2,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_827_TX_2_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_2,TX,0,0,0,0,...,0,0,0,0,0,0,1,0,1,12
30486,FOODS_3_827_TX_3_evaluation,FOODS_3_827,FOODS_3,FOODS,TX_3,TX,0,0,0,0,...,0,2,2,2,1,0,4,1,2,12
30487,FOODS_3_827_WI_1_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_1,WI,0,0,0,0,...,6,8,1,2,0,1,5,5,10,12
30488,FOODS_3_827_WI_2_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_2,WI,0,0,0,0,...,3,3,0,2,0,1,3,4,4,12


In [49]:
train_clu.to_csv(raw_data_dir+'sales_train_item_tskm_eu_evaluation.csv')

### Dept & Store

In [128]:
mean_by_dept_store = train_df.drop(['id', 'item_id', 'cat_id', 'state_id'], axis=1)
mean_by_dept_store = mean_by_dept_store.groupby([mean_by_dept_store['dept_id'], mean_by_dept_store['store_id']]).mean()

In [129]:
mean_by_dept_store = mean_by_dept_store.reset_index()
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [130]:
a = mean_by_dept_store.drop(['dept_id', 'store_id'], axis=1)
a

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,1.134259,0.814815,...,1.907407,2.013889,1.921296,1.282407,1.398148,1.837963,1.527778,1.435185,1.976852,1.546296
1,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,1.449074,1.259259,...,3.162037,2.907407,2.768519,2.166667,1.800926,1.796296,1.870370,2.703704,3.745370,3.402778
2,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,1.870370,1.342593,...,2.555556,2.300926,2.615741,1.745370,1.768519,1.597222,1.888889,2.115741,2.657407,3.013889
3,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,0.402778,0.305556,...,1.314815,1.361111,1.120370,1.324074,1.125000,1.129630,1.027778,1.296296,1.319444,1.486111
4,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,0.550926,0.439815,...,1.106481,1.300926,1.462963,1.208333,1.087963,1.115741,1.018519,1.384259,1.291667,1.842593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,0.365049,0.285437,...,0.396117,0.458252,0.518447,0.359223,0.330097,0.341748,0.324272,0.493204,0.477670,0.533981
66,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,0.262136,0.182524,...,0.376699,0.456311,0.438835,0.355340,0.394175,0.304854,0.337864,0.392233,0.413592,0.427184
67,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,0.186408,0.163107,...,0.299029,0.452427,0.456311,0.271845,0.318447,0.267961,0.339806,0.349515,0.485437,0.477670
68,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,0.122330,0.128155,...,0.299029,0.357282,0.343689,0.281553,0.267961,0.304854,0.384466,0.337864,0.372816,0.380583


In [114]:
# euclidean
tskm_10 = TimeSeriesKMeans(n_clusters=10, verbose=False, random_state=123, n_jobs=-1)
y_pred_10 = tskm_10.fit_predict(a)
tskm_eu = pd.DataFrame({'tskm_10': y_pred_10})
tskm_eu

In [131]:
mean_by_dept_store = pd.concat([mean_by_dept_store, tskm_eu], axis = 1)
mean_by_dept_store

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_5,tskm_7,tskm_10,tskm_30,tskm_50
0,FOODS_1,CA_1,1.375000,1.314815,0.990741,0.810185,0.842593,0.884259,1.037037,1.217593,...,1.837963,1.527778,1.435185,1.976852,1.546296,1,5,9,18,18
1,FOODS_1,CA_2,1.879630,1.888889,1.101852,1.111111,1.018519,1.217593,1.407407,1.634259,...,1.796296,1.870370,2.703704,3.745370,3.402778,1,1,1,24,24
2,FOODS_1,CA_3,1.578704,1.509259,1.203704,1.069444,1.018519,1.194444,1.319444,1.430556,...,1.597222,1.888889,2.115741,2.657407,3.013889,1,1,1,14,14
3,FOODS_1,CA_4,0.523148,0.574074,0.347222,0.449074,0.495370,0.388889,0.462963,0.513889,...,1.129630,1.027778,1.296296,1.319444,1.486111,2,2,2,21,39
4,FOODS_1,TX_1,0.703704,0.712963,0.504630,0.583333,0.500000,0.708333,0.333333,0.583333,...,1.115741,1.018519,1.384259,1.291667,1.842593,2,2,2,21,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0.386408,0.355340,0.227184,0.209709,0.190291,0.213592,0.258252,0.388350,...,0.341748,0.324272,0.493204,0.477670,0.533981,0,0,0,0,0
66,HOUSEHOLD_2,TX_3,0.233010,0.267961,0.188350,0.161165,0.089320,0.198058,0.233010,0.273786,...,0.304854,0.337864,0.392233,0.413592,0.427184,0,0,0,0,0
67,HOUSEHOLD_2,WI_1,0.287379,0.293204,0.128155,0.091262,0.000000,0.203883,0.244660,0.361165,...,0.267961,0.339806,0.349515,0.485437,0.477670,0,0,0,0,0
68,HOUSEHOLD_2,WI_2,0.190291,0.182524,0.133981,0.155340,0.062136,0.207767,0.194175,0.215534,...,0.304854,0.384466,0.337864,0.372816,0.380583,0,0,0,0,0


In [132]:
dcol=[]
for i in list(mean_by_dept_store):
    if 'd_' in i:
        dcol.append(i)
        
mean_by_dept_store = mean_by_dept_store.drop(dcol, axis=1)
mean_by_dept_store

,dept_id,store_id,tskm_5,tskm_7,tskm_10,tskm_30,tskm_50
0,FOODS_1,CA_1,1,5,9,18,18
1,FOODS_1,CA_2,1,1,1,24,24
2,FOODS_1,CA_3,1,1,1,14,14
3,FOODS_1,CA_4,2,2,2,21,39
4,FOODS_1,TX_1,2,2,2,21,21
...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,0,0,0,0,0
66,HOUSEHOLD_2,TX_3,0,0,0,0,0
67,HOUSEHOLD_2,WI_1,0,0,0,0,0
68,HOUSEHOLD_2,WI_2,0,0,0,0,0


In [133]:
train_clu = pd.merge(train_df, mean_by_dept_store, on=['dept_id', 'store_id'])
train_clu

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1937,d_1938,d_1939,d_1940,d_1941,tskm_5,tskm_7,tskm_10,tskm_30,tskm_50
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,3,3,0,1,2,2,2,2,2
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,2,2,2,2,2
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,3,0,1,2,2,2,2,2
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,2,6,2,2,2,2,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,0,1,1,4,4,5,15,15
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,0,1,0,4,4,5,15,15
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,0,1,0,2,4,4,5,15,15
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,1,1,0,4,4,5,15,15


In [138]:
train_clu.to_csv(raw_data_dir+'sales_train_deptstore_tskm_eu_evaluation.csv')